<H1>Analyzing Historical Stock Revenue Data and Building a Dashboard<H1>

In [44]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

we define the function `make_graph`. You should only care about the inputs. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.

In [45]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

<h3>Question 1 - Extracting Tesla Stock Data Using yfinance<h3>

Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.

In [46]:
tesla = yf.Ticker("TSLA")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to `max`.

In [47]:
tesla_data = tesla.history(period="max")

Reset the index, save, and display the first five rows of the `tesla_data` dataframe ussing the `head` function.

In [48]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


<H3>Question 2 - Extracting Tesla Revenue Data Using Webscraping<H3>

We will use the request library for sending an HTTP request to the web page.

In [49]:
url_tesla = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

Now we will use the `request.get()` method which takes the URL and stores it as a variable. We make sure to use .text to save the HTML content as a string in order to make it readable.

In [50]:
html_tesla_data=requests.get(url_tesla).text

Now we will parse the data using `BeautifulSoup`

In [51]:
soup = BeautifulSoup(html_tesla_data, 'html5lib')

Now we will create an empty data frame using the `pd.DataFrame()` function with the following columns:

<ul>
    <li>Date</li>
    <li>Revenue</li>
</ul>

In [52]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

Next, we will focus on the table's main section that holds all the data. We will then iterate through each row to extract the values from every column.

In [83]:
rows_list = []
for row in soup.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text.strip()
    revenue = col[1].text.strip().replace('$', '').replace(',', '')
    rows_list.append({"Date":date, "Revenue":revenue})

tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame(rows_list)], ignore_index=True)
tesla_revenue

,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757
...,...,...
427,2010-06-30,28
428,2010-03-31,21
429,2009-12-31,
430,2009-09-30,46


Now we can print out the data frame using the `head()` or `tail()` function.

In [84]:
tesla_revenue.tail()

,Date,Revenue
427,2010-06-30,28
428,2010-03-31,21
429,2009-12-31,
430,2009-09-30,46
431,2009-06-30,27


<h3>Question 3 - Extracting GameStop Stock Data Using yfinance<h3>

Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.

In [55]:
gamestop = yf.Ticker("GME")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gamestop_data`. Set the `period` parameter to `max`.

In [56]:
gamestop_data = gamestop.history(period="max")

Reset the index, save, and display the first five rows of the `gamestop_data` dataframe using the `head` function.

In [57]:
gamestop_data.reset_index(inplace=True)
gamestop_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693351,1.603297,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603295,1.662209,6892800,0.0,0.0


<H3>Question 4 - Extracting GameStop Revenue Data Using Webscraping<H3>

We will use the request library for sending an HTTP request to the web page.

In [58]:
url_gamestop = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

Now we will use the `request.get()` method which takes the URL and stores it as a variable. We make sure to use .text to save the HTML content as a string in order to make it readable.

In [59]:
html_gamestop_data=requests.get(url_gamestop).text

Now we will parse the data using `BeautifulSoup`

In [60]:
soup_gamestop = BeautifulSoup(html_gamestop_data, 'html5lib')

Now we will create an empty data frame using the `pd.DataFrame()` function with the following columns:

<ul>
    <li>Date</li>
    <li>Revenue</li>
</ul>

In [61]:
gamestop_revenue = pd.DataFrame(columns=["Date", "Revenue"])

Next, we will focus on the table's main section that holds all the data. We will then iterate through each row to extract the values from every column.

In [62]:
tm_rows_list = []
for row in soup_gamestop.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text.strip().replace('$', '').replace(',', '')
    tm_rows_list.append({"Date":date, "Revenue":revenue})

gamestop_revenue = pd.concat([gamestop_revenue, pd.DataFrame(tm_rows_list)], ignore_index=True)

Now we can print out the data frame using the `head()` or `tail()` function.

In [63]:
gamestop_revenue.tail()

,Date,Revenue
11,2009,8806
12,2008,7094
13,2007,5319
14,2006,3092
15,2005,1843


<H3>Question 5 - Tesla Stock and Revenue Dashboard<H3>

Use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(tesla_data, tesla_revenue, 'Tesla')`. 

In [89]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

C:\Users\SpicyRon\AppData\Local\Temp\ipykernel_12848\2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\SpicyRon\AppData\Local\Temp\ipykernel_12848\2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



ValueError: could not convert string to float: ''

<H3>Question 6 - Gamestop Stock and Revenue Dashboard<H3>

Use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(gamestop_data, gamestop_revenue, 'GameStop')`. 

In [ ]:
make_graph(gamestop_data, gamestop_revenue, 'GameStop')

C:\Users\SpicyRon\AppData\Local\Temp\ipykernel_12848\2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\SpicyRon\AppData\Local\Temp\ipykernel_12848\2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



<H3>Author</H3>
   
 By Nooron Ahmad